In [107]:
import cv2
import math
import numpy as np
from  ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [108]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 2050


In [109]:
model_path = r"C:\Users\Rishabh Surana\Desktop\ATMS project\Yolo models\yolov8m.pt"
video_path = r"C:\Users\Rishabh Surana\Desktop\ATMS project\test_data\test_video\video 2.avi"

In [110]:
model = YOLO(model_path)

In [111]:
tracker = DeepSort(max_age=30)

In [112]:
cap = cv2.VideoCapture(video_path)

In [113]:
if not cap.isOpened():
    print("Error : Could not open video")
    exit()
else:
    print("Video is opened and working")

Video is opened and working


In [114]:
meters_per_pixel = 5/50
print(meters_per_pixel)

0.1


In [115]:
def obj_Detection(results, detections):
    for box in results.boxes:
        x1, y1, x2, y2 = map(int,box.xyxy[0])
        class_id = int(box.cls[0])
        class_name = results.names[class_id]
        confidence = float(box.conf[0])
        
        detections.append(([x1,y1,x2-x1,y2-y1], confidence, class_name))

In [116]:
stall_cars = []
def stall_movement_fn(car_coordinates, car_id):
    def eucledian(base_coord, pt):
        pt1 = (base_coord[0] - pt[0])**2
        pt2 = (base_coord[1] - pt[1])**2
        dist = math.sqrt(pt1+pt2)
        return dist

    coords = car_coordinates[car_id]
    stalled = True
    if(len(coords) > 10):
        recent_coords = coords[-10:]
        base_coord = recent_coords[0]
        for pt in recent_coords[1:]:
            distance = eucledian(base_coord, pt)
            if distance >= 5: #pixel_threshold
                stalled = False
                break
    else:
        stalled = False
        
    return stalled
    
    

In [117]:
def alert(car_id, reason):
    if(reason=="Opposite_direction"):
        print(f"{car_id} is moving in Opposite Direction.")
    elif(reason=="Stalled"):
        print(f"{car_id} is stalled.")
    elif(reason=="Overspeed"):
        print(f"{car_id} is Overspeeding.")
    else:
        print(f"{car_id} is ok")

In [118]:
opposite_cars = []
def detect_car_movement(car_coordinates, expected = [0,-1], threshold = 0.4):
    
    def normalize_vector(v):
        mag = math.sqrt(v[0]**2 + v[1]**2)
        if(mag!=0):
            return [v[0]/mag, v[1]/mag]
        else:
            return [0,0]
        
    def cosine_similarity(movment_vec, expected):
        dot_num = (movment_vec[0] * expected[0]) + (movment_vec[1] * expected[1])
        mag1 = math.sqrt((movment_vec[0]**2)+(movment_vec[1]**2))
        mag2 = math.sqrt((expected[0]**2)+(expected[1]**2))
        if(mag1 and mag2):
            return dot_num / (mag1 * mag2)
        else:
            return 1
    
    for car_id, coords in car_coordinates.items():
        
        if(len(coords)<2):
            continue
        
        x1, y1 = coords[-2] 
        x2, y2 = coords[-1]
        
        if(x2-x1 == 0 and y2-y1 == 0):
            if(stall_movement_fn(car_coordinates, car_id)):
                if(car_id not in stall_cars):
                    stall_cars.append(car_id)
                    alert(car_id,"Stalled")
        movment = [x2-x1, y2-y1]
        movement_unit = normalize_vector(movment)
        similarity = cosine_similarity(movement_unit, expected)
        
        
        if(similarity < -threshold):
            if(car_id not in opposite_cars):
                alert(car_id,"Opposite_direction")
                opposite_cars.append(car_id)
        

In [119]:
def calculate_speed(car_coordinates, fps, meters_per_pixel):
    speeds = []
    for i in range(1, len(car_coordinates)):
        x1, y1 = car_coordinates[i-1]
        x2, y2 = car_coordinates[i]
        pixel_dist = math.hypot(x2 - x1, y2 - y1)
        distance_m = pixel_dist * meters_per_pixel
        time_s = 1 / fps
        speed_mps = distance_m / time_s
        speed_kmph = speed_mps * 3.6
        speeds.append(speed_kmph)
    if speeds:
        return sum(speeds)/len(speeds)
    else:
        return 0.0

In [120]:
car_coordinates = {}
def car_coords_fn(track_id, x1, y1, x2, y2):
    center_x = int((x1 + x2)/2)    
    center_y = int((y1 + y2)/2)
    if track_id not in car_coordinates:
        car_coordinates[track_id] = []
    if len(car_coordinates[track_id]) >= 20:
        car_coordinates[track_id].pop(0)
    car_coordinates[track_id].append([center_x,center_y])
    detect_car_movement(car_coordinates)

In [121]:
overspeed = {}
def draw_Tracks(tracks,frame):
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        class_name = track.get_det_class()
        
        car_coords_fn(track_id, x1, y1, x2, y2)
        speed = calculate_speed(car_coordinates[track_id], fps = 30, meters_per_pixel=meters_per_pixel)
        if(speed > 60 and track_id not in overspeed):
            alert(track_id,"Overspeed")
            overspeed[track_id] = speed
            
        label = f"ID {track_id} | {class_name} | {speed:.1f} km/h"
        
        if track_id in stall_cars:
            color = (0,255,255)
        elif track_id in opposite_cars:
            color = (0,0,255)
        elif speed > 60:
            color = (0,165,255)
        else:
            color = (0,255,0)
        
        
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 1)
        cv2.putText(frame, label, (x1, y1 - 10,), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
        

In [122]:
while True:
   ret, frame = cap.read()
   if not ret:
       break
#    frame = bev_func(frame)
   results = model(frame, conf = 0.6)[0]
   detections = []
   
   obj_Detection(results, detections)
   
   tracks = tracker.update_tracks(detections, frame = frame)
   
   draw_Tracks(tracks,frame)
   
   cv2.imshow("Output", frame)
   if(cv2.waitKey(1) and 0xFF == ord('q')):
        break

cap.release()
cv2.destroyAllWindows()
   
   


0: 480x640 3 cars, 1 truck, 38.2ms
Speed: 2.6ms preprocess, 38.2ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 33.9ms
Speed: 2.8ms preprocess, 33.9ms inference, 6.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 34.1ms
Speed: 2.3ms preprocess, 34.1ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 33.7ms
Speed: 1.9ms preprocess, 33.7ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 1 truck, 32.3ms
Speed: 2.4ms preprocess, 32.3ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 33.9ms
Speed: 2.8ms preprocess, 33.9ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 32.8ms
Speed: 1.8ms preprocess, 32.8ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 32.4ms
Speed: 2.2ms preprocess, 32.4ms inference, 4.0ms postprocess per image at shape (1, 3, 480

In [123]:
total_cars = len(car_coordinates)
print(f"Total Cars = {total_cars}")
print("--"*130)

for track_id, coords in car_coordinates.items():
    print(f"\nCar Id {track_id} -> {coords}")
    speed = calculate_speed(coords, fps = 30, meters_per_pixel=meters_per_pixel)
    print(f"Speed of car Id {track_id} : {speed:.2f} km/h.")

Total Cars = 13
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Car Id 1 -> [[169, 108], [168, 106], [167, 104], [166, 102], [166, 100], [165, 98], [164, 96], [163, 94], [162, 93], [161, 91], [161, 89], [160, 87], [159, 85], [158, 83], [157, 81], [156, 79], [156, 78], [155, 76], [154, 74], [153, 72]]
Speed of car Id 1 : 22.71 km/h.

Car Id 4 -> [[148, 107], [148, 105], [148, 104], [147, 102], [147, 100], [146, 99], [146, 97], [146, 95], [145, 93], [145, 91], [145, 89], [144, 88], [144, 86], [143, 84], [143, 82], [143, 80], [143, 78], [142, 76], [142, 75], [141, 73]]
Speed of car Id 4 : 20.47 km/h.

Car Id 9 -> [[181, 136], [180, 133], [179, 130], [178, 128], [176, 126], [180, 127], [179, 126], [179, 124], [178, 122], [177, 120], [177, 118], [176, 116], [175, 114], [174, 112],

In [124]:
print("\n---Stalled Vehicle---\n")
for s_car in stall_cars:
    print(f"Car Id {s_car} Stalled.")
print("\n\n---Opposite Vehicle---\n")
for op_car in opposite_cars:
    print(f"Car Id {op_car} is moving in opposite direction.")
print("\n\n---Overspeed Vehicle---\n")
for overspd in overspeed:
    print(f"Car Id {overspd} is overspeeding.")


---Stalled Vehicle---

Car Id 38 Stalled.


---Opposite Vehicle---

Car Id 9 is moving in opposite direction.


---Overspeed Vehicle---

Car Id 9 is overspeeding.
Car Id 13 is overspeeding.
